In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU')) #"/GPU:0": The first GPU of machine that is visible to TensorFlow.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.test.is_built_with_cuda()
# tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns+
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")
print("Loaded")

## Load the Dataset

In [ ]:
# tf.debugging.set_log_device_placement(True) 
corpus = pd.read_csv("super23_train.csv", encoding='latin-1')
corpus.columns =["sentence","label"]
# corpus['label']= corpus['label'].map({'ham': 0, 'spam': 1})
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

In [ ]:
corpus.info()
corpus.groupby(by='label').count()
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
# sentences[0]

In [ ]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

In [ ]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [ ]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model11 = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model11.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model11

In [ ]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [ ]:
import time

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activation = ['relu']
filters = 100
kernel_size = [3]

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
print(sentences[0])

train_x = sentences
train_y = labels

# Turn the labels into a numpy array
train_y = np.array(train_y)

# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1
            
# Define the input shape
model11 = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

# Train the model and initialize test accuracy with 0
acc = 0
start = time.time()

model11.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=10, verbose=1, callbacks=[callbacks])

stop = time.time()
print(f"Training time: {stop - start}s")           
# mean_acc = np.array(acc_list).mean()
# parameters = [activation, kernel_size]
# entries = parameters + acc_list + [mean_acc]

# temp = pd.DataFrame([entries], columns=columns)
# record = record.append(temp, ignore_index=True)
# print()
# print(record)

In [ ]:
corpus1 = pd.read_csv("super23_test.csv", encoding='latin-1')
# corpus1 = pd.read_excel("test_punny.xlsx")
corpus1.columns =["sentence", "label"]
# corpus['label']= corpus['label'].map({'ham': 0, 'spam': 1})
corpus1.label = corpus1.label.astype(int)
print(corpus1.shape)

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)
print(sentences1[0])

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)

test_x1 = sentences1
test_y1 = labels1

test_sequences1 = tokenizer.texts_to_sequences(test_x1)
Xtest1 = pad_sequences(test_sequences1, maxlen=max_len, padding=padding_type, truncating=trunc_type)

print("Messages length", len(test_x1))

(13148, 2)
download whichapp for whatsapp friends to see your friends apps and also save battery by  you have  friend waiting httpbitlyogmdkv
Messages length 13148


In [ ]:
start = time.time()
pred_lbl = (model11.predict([Xtest1,Xtest1])> 0.5).astype("int32")
stop = time.time()
print(f"classification time: {stop - start}s")
pd.DataFrame(pred_lbl).to_csv('cda7.csv', index=False)

#pd.DataFrame(Xtest).to_csv('xtest.csv', index=False) # numpy array to CSV
import pandas as pd
messages = pd.read_csv("cda7.csv", encoding='latin-1')
messages.columns = ["labels"]
print (messages.tail(3))

from sklearn import metrics
# print(metrics.classification_report(labels1, messages["labels"]))
print(metrics.confusion_matrix(labels1, messages["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(labels1, messages["labels"])))

# Model 2: Word2Vec Static

In [ ]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
!wget -c "https://figshare.com/ndownloader/files/10798046"

In [ ]:
!gunzip ./GoogleNews-vectors-negative300.bin.gz

In [ ]:
# !pip install gensim

In [ ]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

2. __Check number of training words present in Word2Vec__

In [ ]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [ ]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

In [ ]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

In [ ]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [ ]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

In [ ]:
def ensemble_CNN_BiGRU(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel BiGRU
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model_0 = ensemble_CNN_BiGRU(input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

## Train and Test the Model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [ ]:
import time
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activation = ['relu']
filters = 100
kernel_size = [3]
emb_mean = emb_mean
emb_std = emb_std

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

train_x = sentences
train_y = labels

# Turn the labels into a numpy array
train_y = np.array(train_y)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
# Define the input shape
model = ensemble_CNN_BiGRU(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

# Train the model and initialize test accuracy with 0
acc = 0
start = time.time()             
# Train the model
model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=10, verbose=1, callbacks=[callbacks])
stop = time.time()
print(f"Training time: {stop - start}s")           
# mean_acc = np.array(acc_list).mean()
# parameters = [activation, kernel_size]
# entries = parameters + acc_list + [mean_acc]

# temp = pd.DataFrame([entries], columns=columns)
# record = record.append(temp, ignore_index=True)
# print()
# print(record)

In [ ]:
corpus1 = pd.read_csv("super23_test.csv", encoding='latin-1')
# corpus1 = pd.read_excel("test_punny.xlsx")
corpus1.columns =["sentence", "label"]
# corpus['label']= corpus['label'].map({'ham': 0, 'spam': 1})
corpus1.label = corpus1.label.astype(int)
print(corpus1.shape)

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)
print(sentences1[0])

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)

test_x1 = sentences1
test_y1 = labels1

test_sequences1 = tokenizer.texts_to_sequences(test_x1)
Xtest1 = pad_sequences(test_sequences1, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print("Messages length", len(test_x1))

(13148, 2)
download whichapp for whatsapp friends to see your friends apps and also save battery by  you have  friend waiting httpbitlyogmdkv
Messages length 13148


In [ ]:
start = time.time()
pred_lbl = (model.predict([Xtest1,Xtest1])> 0.5).astype("int32")
stop = time.time()
print(f"classification time: {stop - start}s")
pd.DataFrame(pred_lbl).to_csv('cda8.csv', index=False)

#pd.DataFrame(Xtest).to_csv('xtest.csv', index=False) # numpy array to CSV
import pandas as pd
messages = pd.read_csv("cda8.csv", encoding='latin-1')
messages.columns = ["labels"]
print (messages.tail(3))

from sklearn import metrics
# print(metrics.classification_report(labels1, messages["labels"]))
print(metrics.confusion_matrix(labels1, messages["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(labels1, messages["labels"])))

411/411 [==============================] - 5s 6ms/step
classification time: 4.811775207519531s
       labels
13145       0
13146       0
13147       0
[[ 1900    18]
 [  938 10292]]
Accuracy of the model : 0.927


In [ ]:
# pred_lbl = (model.predict([Xtest,Xtest]) > 0.5).astype("int32")
# pd.DataFrame(pred_lbl).to_csv('output1.csv', index=False)

In [ ]:
#pd.DataFrame(Xtest).to_csv('xtest.csv', index=False) # numpy array to CSV
# import pandas as pd
# messages = pd.read_csv("output1.csv", encoding='latin-1')
# messages.columns = ["labels"]
# print (messages.tail(3))

In [ ]:
# from sklearn import metrics
# print(metrics.classification_report(test_y, messages["labels"]))
# print(metrics.confusion_matrix(test_y, messages["labels"]))

# # Printing the Overall Accuracy of the model
# print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(test_y, messages["labels"])))

## Summary

In [ ]:
# record2.sort_values(by='AVG', ascending=False)

In [ ]:
# record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

# Model 3: Word2Vec - Dynamic

In [ ]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
# model_0.summary()

## Train the Model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_size = [3]
emb_mean = emb_mean
emb_std = emb_std

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

train_x = sentences
train_y = labels

# Turn the labels into a numpy array
train_y = np.array(train_y)

# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
# Define the input shape
model3 = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

# Train the model and initialize test accuracy with 0
acc = 0
start = time.time()             
# Train the model
model3.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=10, verbose=1, callbacks=[callbacks])
stop = time.time()
print(f"Training time: {stop - start}s")           
# mean_acc = np.array(acc_list).mean()
# parameters = [activation, kernel_size]
# entries = parameters + acc_list + [mean_acc]

# temp = pd.DataFrame([entries], columns=columns)
# record = record.append(temp, ignore_index=True)
# print()
# print(record)

## Test The Model

In [ ]:
corpus1 = pd.read_csv("super23_test.csv", encoding='latin-1')
# corpus1 = pd.read_excel("test_punny.xlsx")
corpus1.columns =["sentence", "label"]
# corpus['label']= corpus['label'].map({'ham': 0, 'spam': 1})
corpus1.label = corpus1.label.astype(int)
print(corpus1.shape)

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)
print(sentences1[0])

# Separate the sentences and the labels
sentences1, labels1 = list(corpus1.sentence), list(corpus1.label)

test_x1 = sentences1
test_y1 = labels1

test_sequences1 = tokenizer.texts_to_sequences(test_x1)
Xtest1 = pad_sequences(test_sequences1, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print("Messages length", len(test_x1))

(13148, 2)
download whichapp for whatsapp friends to see your friends apps and also save battery by  you have  friend waiting httpbitlyogmdkv
Messages length 13148


In [ ]:
start = time.time()
pred_lbl = (model3.predict([Xtest1,Xtest1])> 0.5).astype("int32")
stop = time.time()
print(f"classification time: {stop - start}s")
pd.DataFrame(pred_lbl).to_csv('cda9.csv', index=False)

#pd.DataFrame(Xtest).to_csv('xtest.csv', index=False) # numpy array to CSV
import pandas as pd
messages = pd.read_csv("cda9.csv", encoding='latin-1')
messages.columns = ["labels"]
print (messages.tail(3))

from sklearn import metrics
# print(metrics.classification_report(labels1, messages["labels"]))
print(metrics.confusion_matrix(labels1, messages["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(labels1, messages["labels"])))

411/411 [==============================] - 5s 6ms/step
classification time: 4.729171276092529s
       labels
13145       0
13146       0
13147       0
[[ 1819    99]
 [ 1004 10226]]
Accuracy of the model : 0.916
